In [14]:
# Plantilla de Pre Procesado
# Cómo importar las librerías
import xlwings  as xw
import numpy    as np
import pandas   as pd
import datetime as dt
import os
import sys
from pathlib import Path
import re


if sys.platform == 'linux' :
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        carpeta_trabajo = Path('/content/drive/Mi unidad/Mis_Pagos/Planilla_Gastos') # Google Colab
else:
    #carpeta_inicio  = Path.home()
    #carpeta_gg = Path("C:/users/debus/GestionGastos")
    #carpeta_trabajo = carpeta_inicio / carpeta_gg
    libro = xw.Book('GGastos_v060422.xlsm')
    periodo = libro.sheets('Menu').range('H2').value
    mes  = periodo[0:2]
    anio = periodo[2:]
    carpeta = Path(r'G:\Mi unidad\Mis_Pagos', anio, periodo)
    Archivo = list(carpeta.glob('CA*$*0020382809*.xls'))

# GUARDAR EL ARCHIVO CA $ 900 0020382809-Movimientos.xls COMO Movi.xlsx
# Archivo = carpeta_trabajo + '/' + 'Movi.xls'  HAY QUE VERLO

# NO PUEDO USAR decimal=',', thousands='.' debido al formato que tiene la columna "Monto" Por ejemplo $1.500,45
# datos = pd.read_excel(Archivo, names=['Fecha', 'Concepto', 'Descripcion', 'Monto', 'NroComprobante'], 
                    # skiprows=5, skipfooter=2, decimal=',', thousands='.', engine='openpyxl')

# =============================================================================================
# LEER ARCHIVO DE MOVIMIENTOS CA PESOS BANCOR y GENERAMOS EL DATAFRAME DEL MES 
#
datos = pd.read_excel(Archivo[0], names=['Fecha', 'Concepto', 'Descripcion', 'Monto', 'NroComprobante'], 
                    skiprows=5, skipfooter=2, engine='xlrd')  
datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y')

datos['Monto'] = datos['Monto'].str.lstrip('$')
datos['Monto'] = datos['Monto'].str.strip()
datos['Monto'] = datos['Monto'].str.replace('.', '', regex=False)  # Elimina los puntos de miles
datos['Monto'] = datos['Monto'].str.replace(',', '.', regex=False) # Reemplaza la coma decimal por el punto decimal
datos['Monto'] = pd.to_numeric(datos['Monto'])
#datos['Monto'] = datos['Monto'].astype(float)

datos['NroComprobante'] = datos['NroComprobante'].astype(str)

archivo_cp = Path(r'G:\Mi unidad\Mis_Pagos', anio, periodo, 'Movi.xlsx')
datos.to_excel(archivo_cp, engine='openpyxl')
Movi = datos.copy()


In [15]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Fecha           52 non-null     datetime64[ns]
 1   Concepto        52 non-null     object        
 2   Descripcion     42 non-null     object        
 3   Monto           52 non-null     float64       
 4   NroComprobante  52 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 2.2+ KB


In [12]:
Movi.head()

,Fecha,Concepto,Descripcion,Monto,NroComprobante
0,2022-02-25,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-1500.00,284329
1,2022-02-25,Compras Maestro POSNet,FRUTOS DE REY,-1078.00,470226
2,2022-02-25,TRANSF.HB.EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",-50000.00,473376
3,2022-02-25,Mantenimiento mundo Bancor,NaN,-1432.22,0
4,2022-02-25,IVA sobre comisión,NaN,-300.77,0


In [18]:
 # ESCRIBIMOS LOS MOVIMIENTOS BANCOR A LA PLANILLA "MES" 
    #planilla = xw.Book.caller().sheets[0]

periodo = libro.sheets('Menu').range('H2').value
mes  = periodo[0:2]
anio = periodo[2:]
Mes_dic = {'01':'Ene', '02':'Feb', '03':'Mar', '04':'Abr', '05':'May', '06':'Jun', '07':'Jul', '08':'Ago', '09':'Sep', '10':'Oct', '11':'Nov', '12':'Dic' }
Mes = Mes_dic[mes]

Mes_lis = ['Falso', 'Ene', 'Feb', 'Mar','Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
if Mes == 'Ene' :
    planilla = 'Menu'
else:
    Mes_ant = Mes_lis[(int(mes) - 1)]
    planilla = Mes_ant + '_Efectivo'

libro.sheets('_xlwings.conf').visible=0  # 0 OCULTA
#libro = xw.Book()
libro.sheets.add(name=Mes, after=libro.sheets(planilla)) # OJO TODAS LAS PLANILLAS SE VAN AGREGAR DESPUES DE ENERO
libro.sheets(Mes).range('A1').options(pd.DataFrame, numbers='0,00', dates=dt.date, expand='table').value = Movi
table = libro.sheets(Mes).tables.add(source=libro.sheets(Mes).range('A1').expand(), name=Mes)

libro.sheets(Mes).autofit()
libro.sheets(Mes).range('A:A').column_width=3   # indice
libro.sheets(Mes).range('B:B').column_width=10  # Fecha
libro.sheets(Mes).range('C:C').column_width=30  # Concepto
libro.sheets(Mes).range('D:D').column_width=40  # Descripcion
libro.sheets(Mes).range('E:E').column_width=12  # Monto
libro.sheets(Mes).range('F:F').column_width=10  # NroComprobante
libro.sheets(Mes).range('E:E').number_format = '0,00'


In [48]:
# =============================================================================================
# GENERAMOS PLANILLA DEBITOS
# ====> Genero el DataFrame
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

debitos['Tarjetas'] = np.nan
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan 

debitos['Impuestos'] = np.nan

debitos['Inversiones'] = np.nan
debitos['Inversiones'] = debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto'] = np.nan

debitos['Extracciones'] = np.nan
debitos['Extracciones'] = debitos.loc[debitos['Concepto'].str.contains('Extracci.n.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Concepto'].str.contains('Extracci.n.+', regex=True, na=False), 'Monto'] = np.nan

del debitos['NroComprobante']

debitos['Comentarios'] = ''

debitos = debitos.rename(columns={'Monto':'Gastos'})

debitos.sort_values(by='Gastos', ascending=False)

v_suma_gastos       = debitos['Gastos'].sum()
v_suma_tarjetas     = debitos['Tarjetas'].sum()
v_suma_impuestos    = debitos['Impuestos'].sum()
v_suma_inversiones  = debitos['Inversiones'].sum()
v_suma_extracciones = debitos['Extracciones'].sum()

# debitos.loc[debitos.index.max()+1, :] = [np.nan, np.nan, np.nan, v_suma_gastos, v_suma_tarjetas, v_suma_impuestos, v_suma_inversiones, v_suma_extracciones, '']

# =====> Genero Planilla
#mes = Movi.loc[0, 'Fecha'].strftime('%b')  # Feb 
mes_deb = Mes + '_Debitos'
libro.sheets.add(name=mes_deb, after=libro.sheets(Mes))
libro.sheets(mes_deb).range('A1').options(pd.DataFrame, numbers='0,00', dates=dt.date, expand='table').value = debitos
table = libro.sheets(mes_deb).tables.add(source=libro.sheets(mes_deb).range('A1').expand(), name=mes_deb)
libro.sheets(mes_deb).autofit()
libro.sheets(mes_deb).range('A:A').column_width=3
libro.sheets(mes_deb).range('B:B').column_width=10
libro.sheets(mes_deb).range('C:C').column_width=30
libro.sheets(mes_deb).range('D:D').column_width=40
libro.sheets(mes_deb).range('E:E').column_width=12
libro.sheets(mes_deb).range('F:F').column_width=12
libro.sheets(mes_deb).range('G:G').column_width=12
libro.sheets(mes_deb).range('H:H').column_width=12
libro.sheets(mes_deb).range('I:I').column_width=12
libro.sheets(mes_deb).range('J:J').column_width=30
libro.sheets(mes_deb).range('E:E').number_format = '0,00'
libro.sheets(mes_deb).range('F:F').number_format = '0,00'
libro.sheets(mes_deb).range('G:G').number_format = '0,00'
libro.sheets(mes_deb).range('H:H').number_format = '0,00'
libro.sheets(mes_deb).range('I:I').number_format = '0,00'



In [46]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

In [30]:
#debitos  = Movi.loc[Movi["Monto"] <= 0, :]
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False), 'Monto']

28   -63777.79
Name: Monto, dtype: float64

In [29]:
debitos.loc[debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']

34   -34645.2
Name: Monto, dtype: float64

In [37]:
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False), 'Monto']

In [44]:
#debitos.sort_values(by='Gastos', ascending=False)

In [47]:
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']

28   -63777.79
34   -34645.20
Name: Monto, dtype: float64

In [49]:
debitos

,Fecha,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
0,2022-02-25,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-1500.00,NaN,NaN,NaN,NaN,
1,2022-02-25,Compras Maestro POSNet,FRUTOS DE REY,-1078.00,NaN,NaN,NaN,NaN,
2,2022-02-25,TRANSF.HB.EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",-50000.00,NaN,NaN,NaN,NaN,
3,2022-02-25,Mantenimiento mundo Bancor,NaN,-1432.22,NaN,NaN,NaN,NaN,
4,2022-02-25,IVA sobre comisión,NaN,-300.77,NaN,NaN,NaN,NaN,
5,2022-02-22,Compras Maestro POSNet,CHICOM SA,-937.50,NaN,NaN,NaN,NaN,
6,2022-02-22,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-780.00,NaN,NaN,NaN,NaN,
7,2022-02-18,Compras Maestro POSNet,MERCADOPAGO*EQUUS,-850.00,NaN,NaN,NaN,NaN,
8,2022-02-18,Compras Maestro POSNet,CLEMENTE FULL SHOP,-590.00,NaN,NaN,NaN,NaN,
9,2022-02-18,Compras Maestro POSNet,DELICIUS,-570.00,NaN,NaN,NaN,NaN,


In [ ]:
c = Movi.loc[Movi["Monto"] > 0, :]
creditos = c.copy()

creditos['Haberes'] = np.nan
creditos['Haberes'] = creditos.loc[creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

creditos['Extras']  = np.nan
creditos['Extras']  = creditos.loc[~creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

creditos['Inversiones'] = np.nan
creditos['Inversiones'] = creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo fijo.+', regex=True, na=False), 'Extras']
creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo fijo.+', regex=True, na=False), 'Extras'] = np.nan

#Crédito operación cambio ME por Plataforma Digital

del creditos['Monto']
del creditos['NroComprobante']

In [51]:
c = Movi.loc[Movi["Monto"] > 0, :]
creditos = c.copy()

In [53]:
creditos['Haberes'] = np.nan
creditos['Haberes'] = creditos.loc[creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

In [57]:
creditos

,Fecha,Concepto,Descripcion,Monto,NroComprobante,Haberes,Extras
20,2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",50000.00,924367,NaN,50000.00
29,2022-02-08,Credito por pago de plazo fijo,NaN,275767.53,970629,NaN,275767.53
35,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,0,3278.00,NaN
36,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,0,189820.76,NaN
39,2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,12000.00,883396,NaN,12000.00
42,2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",18200.00,268469,NaN,18200.00
50,2022-02-01,Interes por capitalización,NaN,6.39,0,NaN,6.39


In [56]:
creditos['Extras']  = np.nan
creditos['Extras']  = creditos.loc[~creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

In [59]:
creditos['Inversiones'] = np.nan
creditos['Inversiones'] = creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo\s+fijo', regex=True, na=False), 'Extras']
creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo\s+fijo', regex=True, na=False), 'Extras'] = np.nan
creditos

,Fecha,Concepto,Descripcion,Monto,NroComprobante,Haberes,Extras,Inversiones
20,2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",50000.00,924367,NaN,50000.00,NaN
29,2022-02-08,Credito por pago de plazo fijo,NaN,275767.53,970629,NaN,NaN,275767.53
35,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,0,3278.00,NaN,NaN
36,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,0,189820.76,NaN,NaN
39,2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,12000.00,883396,NaN,12000.00,NaN
42,2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",18200.00,268469,NaN,18200.00,NaN
50,2022-02-01,Interes por capitalización,NaN,6.39,0,NaN,6.39,NaN


In [60]:
del creditos['Monto']
del creditos['NroComprobante']

In [61]:
creditos

,Fecha,Concepto,Descripcion,Haberes,Extras,Inversiones
20,2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",NaN,50000.00,NaN
29,2022-02-08,Credito por pago de plazo fijo,NaN,NaN,NaN,275767.53
35,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,NaN,NaN
36,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,NaN,NaN
39,2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,NaN,12000.00,NaN
42,2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",NaN,18200.00,NaN
50,2022-02-01,Interes por capitalización,NaN,NaN,6.39,NaN


In [ ]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

debitos['Tarjetas'] = np.nan
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan 

In [63]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

In [65]:
debitos['Tarjetas'] = np.nan

In [68]:
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
debitos

,Fecha,Concepto,Descripcion,Monto,NroComprobante,Tarjetas
0,2022-02-25,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-1500.00,284329,NaN
1,2022-02-25,Compras Maestro POSNet,FRUTOS DE REY,-1078.00,470226,NaN
2,2022-02-25,TRANSF.HB.EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",-50000.00,473376,NaN
3,2022-02-25,Mantenimiento mundo Bancor,NaN,-1432.22,0,NaN
4,2022-02-25,IVA sobre comisión,NaN,-300.77,0,NaN
5,2022-02-22,Compras Maestro POSNet,CHICOM SA,-937.50,22209,NaN
6,2022-02-22,Compras Maestro POSNet,VIUMI*CARNES EL FARAON,-780.00,773913,NaN
7,2022-02-18,Compras Maestro POSNet,MERCADOPAGO*EQUUS,-850.00,761673,NaN
8,2022-02-18,Compras Maestro POSNet,CLEMENTE FULL SHOP,-590.00,844443,NaN
9,2022-02-18,Compras Maestro POSNet,DELICIUS,-570.00,56977,NaN


In [70]:
c = Movi.loc[Movi["Monto"] > 0, :]
creditos = c.copy()

creditos['Haberes'] = np.nan
creditos['Haberes'] = creditos.loc[creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

creditos['Extras']  = np.nan
creditos['Extras']  = creditos.loc[~creditos['Concepto'].str.contains('Acreditaci.n.+Haberes.+', regex=True, na=False), 'Monto']

#Credito por pago de plazo fijo | Crédito operación cambio ME por Plataforma Digital
creditos['Inversiones'] = np.nan
creditos['Inversiones'] = creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo\s+fijo', regex=True, na=False) | creditos['Concepto'].str.contains('Cr.dito.+cambio\sME.+', regex=True, na=False), 'Extras']
creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+plazo\s+fijo', regex=True, na=False) | creditos['Concepto'].str.contains('Cr.dito.+cambio\sME.+', regex=True, na=False), 'Extras'] = np.nan

creditos.loc[creditos['Concepto'].str.contains('Cr.dito.+cambio\sME.+', regex=True, na=False), 'Extras'] = np.nan

In [71]:
creditos

,Fecha,Concepto,Descripcion,Monto,NroComprobante,Haberes,Extras,Inversiones
20,2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",50000.00,924367,NaN,50000.00,NaN
29,2022-02-08,Credito por pago de plazo fijo,NaN,275767.53,970629,NaN,NaN,275767.53
35,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,0,3278.00,NaN,NaN
36,2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,0,189820.76,NaN,NaN
39,2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,12000.00,883396,NaN,12000.00,NaN
42,2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",18200.00,268469,NaN,18200.00,NaN
50,2022-02-01,Interes por capitalización,NaN,6.39,0,NaN,6.39,NaN


In [74]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

debitos['Tarjetas'] = np.nan
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan 

debitos['Impuestos'] = np.nan

debitos['Inversiones'] = np.nan
debitos['Inversiones'] = debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto'] = np.nan

debitos['Extracciones'] = np.nan
debitos['Extracciones'] = debitos.loc[debitos['Concepto'].str.contains('Extracci.n.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Concepto'].str.contains('Extracci.n.+', regex=True, na=False), 'Monto'] = np.nan

del debitos['NroComprobante']

debitos['Comentarios'] = ''

debitos = debitos.rename(columns={'Monto':'Gastos'})

debitos_ord = debitos.sort_values(by='Gastos', ascending=False)

In [110]:
d = Movi.loc[Movi["Monto"] <= 0, :]
debitos = d.copy()

In [ ]:
#debitos['Inversiones'] = np.nan
debitos['Inversiones'] = debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto']
debitos.loc[debitos['Concepto'].str.contains('D.bito plazo fijo.+', regex=True, na=False) | debitos['Concepto'].str.contains('D.bito operaci.n.+cambio ME.+', regex=True, na=False) | debitos['Concepto'].str.contains('IMPUESTO P.A.I.S.+', regex=True, na=False) | debitos['Concepto'].str.contains('.+GAN.+TENENCIA.+4815/20.+', regex=True, na=False), 'Monto'] = np.nan


In [79]:
debitos = d.copy()


debitos['Tarjetas'] = np.nan
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']
#debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan 

In [112]:
debitos['Tarjetas'] = np.nan

In [87]:
debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']

28   -63777.79
34   -34645.20
Name: Monto, dtype: float64

In [128]:
debitos.loc[indice.index, 'Tarjetas'] = indice

In [126]:
indice = debitos.loc[debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']

In [130]:
debitos.loc[indice.index, 'Monto'] = np.nan

In [103]:
debitos.loc[debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto'] = np.nan

In [106]:
a = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False), 'Monto']

In [107]:
a.info()

<class 'pandas.core.series.Series'>
Int64Index: 1 entries, 28 to 28
Series name: Monto
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 16.0 bytes


In [97]:
debitos['Tarjetas'] = debitos.loc[debitos['Descripcion'].str.contains('TARJNARANJA.+', regex=True, na=False) | debitos['Descripcion'].str.contains('VISA.+', regex=True, na=False), 'Monto']

34   -34645.2
Name: Monto, dtype: float64

In [135]:
libro = xw.Book('GGastos_v060422.xlsm')

In [136]:
debitos = libro.sheets(mes_deb).range('B1').options(pd.DataFrame, expand='table').value
v_suma_gastos_deb       = debitos['Gastos'].sum()
v_suma_tarjetas_deb     = debitos['Tarjetas'].sum()
v_suma_impuestos_deb    = debitos['Impuestos'].sum()
v_suma_inversiones_deb  = debitos['Inversiones'].sum()
v_suma_extracciones_deb = debitos['Extracciones'].sum()


In [137]:
debitos

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
Fecha,,,,,,,,
2022-02-01,IVA sobre comisión,None,-1.55,NaN,NaN,NaN,NaN,None
2022-02-01,Interés saldo deudor cuenta adelanto,None,-7.38,NaN,NaN,NaN,NaN,None
2022-02-02,Compras Maestro POSNet,MERCADOPAGO*FARMACIASO,-180.00,NaN,NaN,NaN,NaN,None
2022-02-09,Compras Maestro POSNet,TIENDAS BAZAR COM SRL,-270.00,NaN,NaN,NaN,NaN,None
2022-02-25,IVA sobre comisión,None,-300.77,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,DSA FCIA CRISTAL,-400.00,NaN,NaN,NaN,NaN,None
2022-02-15,Compras Maestro POSNet,PANIFICADORA PUGLIESE,-460.00,NaN,NaN,NaN,NaN,None
2022-02-01,Compras Maestro POSNet,MERCADOPAGO*NATYVOS,-470.00,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,MERCADOPAGO*AIRESDELAS,-550.00,NaN,NaN,NaN,NaN,None


In [143]:
debitos.loc[v_Fecha, :] = ['Total :', '', v_suma_gastos_deb, v_suma_tarjetas_deb, v_suma_impuestos_deb, v_suma_inversiones_deb, v_suma_extracciones_deb, '']

In [141]:
ult_dia = str(calendar.monthrange(int(anio), int(mes))[1])
v_Fecha = anio + '-' + mes + '-' + ult_dia 

In [140]:
import calendar

In [142]:
v_Fecha

'2022-02-28'

In [145]:
debitos

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
Fecha,,,,,,,,
2022-02-01,IVA sobre comisión,None,-1.55,NaN,NaN,NaN,NaN,None
2022-02-01,Interés saldo deudor cuenta adelanto,None,-7.38,NaN,NaN,NaN,NaN,None
2022-02-02,Compras Maestro POSNet,MERCADOPAGO*FARMACIASO,-180.00,NaN,NaN,NaN,NaN,None
2022-02-09,Compras Maestro POSNet,TIENDAS BAZAR COM SRL,-270.00,NaN,NaN,NaN,NaN,None
2022-02-25,IVA sobre comisión,None,-300.77,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,DSA FCIA CRISTAL,-400.00,NaN,NaN,NaN,NaN,None
2022-02-15,Compras Maestro POSNet,PANIFICADORA PUGLIESE,-460.00,NaN,NaN,NaN,NaN,None
2022-02-01,Compras Maestro POSNet,MERCADOPAGO*NATYVOS,-470.00,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,MERCADOPAGO*AIRESDELAS,-550.00,NaN,NaN,NaN,NaN,None


In [148]:
mes_cre = 'Feb_Creditos'

In [149]:
creditos = libro.sheets(mes_cre).range('B1').options(pd.DataFrame, expand='table').value



In [150]:
creditos

,Concepto,Descripcion,Haberes,Extras,Inversiones
Fecha,,,,,
2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,NaN,NaN
2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,NaN,NaN
2022-02-01,Interes por capitalización,None,NaN,6.39,NaN
2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,NaN,12000.00,NaN
2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",NaN,18200.00,NaN
2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",NaN,50000.00,NaN
2022-02-08,Credito por pago de plazo fijo,None,NaN,NaN,275767.53
2022-02-08,Calculo interes,None,NaN,NaN,-250000.00


In [151]:
v_suma_haberes = creditos['Haberes'].sum()
v_suma_extras  = creditos['Extras'].sum()
v_suma_inversiones  = creditos['Inversiones'].sum()

creditos.loc[v_Fecha, :] = ['Total :', '', v_suma_haberes, v_suma_extras, v_suma_inversiones]


In [152]:
creditos

,Concepto,Descripcion,Haberes,Extras,Inversiones
Fecha,,,,,
2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,3278.00,NaN,NaN
2022-02-04,Acreditación de Haberes TA,EPEC - HABERES,189820.76,NaN,NaN
2022-02-01,Interes por capitalización,None,NaN,6.39,NaN
2022-02-02,CREDIN - GRAVADO SIRCREB,20343702087-Ezequiel Arrieta Ponce-,NaN,12000.00,NaN
2022-02-02,TRANSF. CREDITO HB -EXENTO,"27423847218-BUSSO, CONSTANZA CAROL-SEG",NaN,18200.00,NaN
2022-02-14,TRANSF. CREDITO HB -EXENTO,"23208706454-NAVAS, MARIA GABRIELA-VAR",NaN,50000.00,NaN
2022-02-08,Credito por pago de plazo fijo,None,NaN,NaN,275767.53
2022-02-08,Calculo interes,None,NaN,NaN,-250000.00
2022-02-28,Total :,,193098.76,80206.39,25767.53


In [158]:

v_suma_gastos_efe       = efectivo['Gastos'].sum()
v_suma_tarjetas_efe     = efectivo['Tarjetas'].sum()
v_suma_impuestos_efe    = efectivo['Impuestos'].sum()
v_suma_inversiones_efe  = efectivo['Inversiones'].sum()
v_suma_extras_efe       = efectivo['Extras'].sum()


In [156]:
mes_efe = 'Feb' + '_Efectivo'
efectivo = libro.sheets(mes_efe).range('A1').options(pd.DataFrame, expand='table').value


In [157]:
efectivo

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extras,Comentarios
Fecha,,,,,,,,
2022-02-01,Varios,None,-279487.0,0.0,0.0,0.0,0.0,None


In [159]:
efectivo.loc[v_Fecha, :] = ['Total :', '', v_suma_gastos_efe, v_suma_tarjetas_efe, v_suma_impuestos_efe, v_suma_inversiones_efe, v_suma_extras_efe, '']

In [160]:
efectivo

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extras,Comentarios
Fecha,,,,,,,,
2022-02-01,Varios,None,-279487.0,0.0,0.0,0.0,0.0,None
2022-02-28,Total :,,-279487.0,0.0,0.0,0.0,0.0,


In [161]:
libro.sheets(mes_efe).range('A1').options(pd.DataFrame, expand='table').value = efectivo

In [187]:
rango = libro.sheets(mes_deb).tables(mes_deb).data_body_range

In [188]:
rango.get_address()

'$A$2:$J$47'

In [193]:
rango

<Range [GGastos_v060422.xlsm]Feb_Debitos!$A$2:$J$47>

In [189]:
rango.get_address(row_absolute=False, column_absolute=False)

'A2:J47'

In [191]:
fila = rango.get_address(row_absolute=False, column_absolute=False)[4:]

In [196]:
str(int(fila) + 3)

'50'

In [200]:
totales = pd.DataFrame(efectivo.loc[v_Fecha, :], index=debito.index)

NameError: name 'Fecha' is not defined

In [202]:
debitos

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
Fecha,,,,,,,,
2022-02-01,IVA sobre comisión,None,-1.55,NaN,NaN,NaN,NaN,None
2022-02-01,Interés saldo deudor cuenta adelanto,None,-7.38,NaN,NaN,NaN,NaN,None
2022-02-02,Compras Maestro POSNet,MERCADOPAGO*FARMACIASO,-180.00,NaN,NaN,NaN,NaN,None
2022-02-09,Compras Maestro POSNet,TIENDAS BAZAR COM SRL,-270.00,NaN,NaN,NaN,NaN,None
2022-02-25,IVA sobre comisión,None,-300.77,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,DSA FCIA CRISTAL,-400.00,NaN,NaN,NaN,NaN,None
2022-02-15,Compras Maestro POSNet,PANIFICADORA PUGLIESE,-460.00,NaN,NaN,NaN,NaN,None
2022-02-01,Compras Maestro POSNet,MERCADOPAGO*NATYVOS,-470.00,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,MERCADOPAGO*AIRESDELAS,-550.00,NaN,NaN,NaN,NaN,None


In [211]:
s = debitos.loc[v_Fecha, :] 

In [225]:
s = debitos.iloc[-1]

In [227]:
s.transpose()

Concepto          Total :
Descripcion              
Gastos         -221598.97
Tarjetas        -93867.99
Impuestos         -4555.0
Inversiones      -18232.5
Extracciones     -80000.0
Comentarios              
Name: 2022-02-28 00:00:00, dtype: object

In [230]:
totales = pd.DataFrame(columns = ['Concepto', 'Descripcion', 'Gastos', 'Tarjetas', 'Impuestos', 'Inversiones', 'Extracciones', 'Comentarios'])

In [234]:
v_suma_tarjetas_sin_imp = v_suma_tarjetas_deb

In [235]:
totales.loc[v_Fecha, :] = ['Total :', '', v_suma_gastos_deb, v_suma_tarjetas_sin_imp, v_suma_impuestos_deb, v_suma_inversiones_deb, v_suma_extracciones_deb, '']

In [236]:
totales

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
2022-02-28,Total :,,-221598.97,-93867.99,-4555.0,-18232.5,-80000.0,


In [228]:
debitos

,Concepto,Descripcion,Gastos,Tarjetas,Impuestos,Inversiones,Extracciones,Comentarios
Fecha,,,,,,,,
2022-02-01,IVA sobre comisión,None,-1.55,NaN,NaN,NaN,NaN,None
2022-02-01,Interés saldo deudor cuenta adelanto,None,-7.38,NaN,NaN,NaN,NaN,None
2022-02-02,Compras Maestro POSNet,MERCADOPAGO*FARMACIASO,-180.00,NaN,NaN,NaN,NaN,None
2022-02-09,Compras Maestro POSNet,TIENDAS BAZAR COM SRL,-270.00,NaN,NaN,NaN,NaN,None
2022-02-25,IVA sobre comisión,None,-300.77,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,DSA FCIA CRISTAL,-400.00,NaN,NaN,NaN,NaN,None
2022-02-15,Compras Maestro POSNet,PANIFICADORA PUGLIESE,-460.00,NaN,NaN,NaN,NaN,None
2022-02-01,Compras Maestro POSNet,MERCADOPAGO*NATYVOS,-470.00,NaN,NaN,NaN,NaN,None
2022-02-16,Compras Maestro POSNet,MERCADOPAGO*AIRESDELAS,-550.00,NaN,NaN,NaN,NaN,None


In [219]:
s2 = pd.DataFrame(s, index=[v_Fecha])

In [220]:
s2

,2022-02-28
2022-02-28,NaN


In [ ]:
s

In [209]:
s.transpose()

Concepto          Total :
Descripcion              
Gastos         -221598.97
Tarjetas        -93867.99
Impuestos         -4555.0
Inversiones      -18232.5
Extracciones     -80000.0
Comentarios              
Name: 2022-02-28 00:00:00, dtype: object

In [210]:
s

Concepto          Total :
Descripcion              
Gastos         -221598.97
Tarjetas        -93867.99
Impuestos         -4555.0
Inversiones      -18232.5
Extracciones     -80000.0
Comentarios              
Name: 2022-02-28 00:00:00, dtype: object

In [241]:
rango = libro.sheets(mes_efe).tables(mes_efe).data_body_range
#fila = rango.get_address(row_absolute=False, column_absolute=False)[4:]
#fila_ini = str(int(fila) + 3)
#rng_ini = 'B'+ fila_ini

fila = rango.get_address(row_absolute=False, column_absolute=False)

KeyError: 'Feb_Efectivo'

In [242]:
libro.sheets(mes_efe).tables(mes_efe).data_body_range

KeyError: 'Feb_Efectivo'

In [ ]:
rng_ini

In [ ]:
fila

In [237]:
mes_efe

'Feb_Efectivo'